<a href="https://colab.research.google.com/github/superyaniv/jobinator/blob/main/engine_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=nF2_IuvMyRWPuVx38-1vtzdXmpFSltw4J1DRWlw4u2A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgfPch_1LFXfPUhEJBHg9tWnvn4shv360uowr1FuU9ie4dBKxo9x0A

Successfully saved authorization token.


In [96]:
%%capture output
!pip install geemap geopandas

In [97]:
import google.colab
import geemap.eefolium as geemap
from IPython.display import Image
import geopandas as gpd
import folium
ee.Initialize()
import ee

In [127]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
kasat = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/kmls/kasat.kml', driver='KML')
usbeams_lg = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/kmls/large_beams.kml', driver='KML')
usbeams_sm = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/kmls/Anik-f2_feb_2016.kml', driver='KML')
# Map = geemap.Map(center=[12,105], zoom= 3)
# dem = ee.Image('USGS/SRTMGL1_003')

# // COUNTRY LIBRARY
# var country_co = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
#   .filter(ee.Filter.eq('country_co', 'FR'));
country_co = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co','FR'))
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012').select('stable_lights').clipToCollection(country_co)

In [139]:
test.addLayer(zones)
# zones.addBands(nl2012).reduceToVectors
# ee.Reducer(ee.Reducer.max)
# var vectors = zones.addBands(nl2012).reduceToVectors({
#   geometry: country_co,
#   crs: nl2012.projection(),
#   scale: 1000,
#   geometryType: 'polygon',
#   eightConnected: false,
#   labelProperty: 'zone',
#   reducer: ee.Reducer.mean()
# });

AttributeError: ignored

In [134]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [135]:
test = folium.Map(location=[55,-40],zoom_start=3, tiles='CartoDB dark_matter')

In [143]:
zones = nl2012.gt(30).add(nl2012.gt(55)).add(nl2012.gt(62))
zones = zones.updateMask(zones.neq(0))

In [144]:
zones.reduceToVectors()

In [137]:
for _, s in kasat.iterrows():
    sim_geo = gpd.GeoSeries(s['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(s['geometry']).add_to(geo_j)
    geo_j.add_to(test)
for _, s in usbeams_lg.iterrows():
    sim_geo = gpd.GeoSeries(s['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'yellow'})
    folium.Popup(s['geometry']).add_to(geo_j)
    geo_j.add_to(test)

In [113]:
display(test)

In [106]:
# // Instantiate an image with the Image constructor.
# var popden = ee.Image('users/yanivalfasy/ppp_2020_1km_Aggregated');
# var eurobeams = ee.FeatureCollection('users/yanivalfasy/kasat')
# var usbeams = ee.FeatureCollection('users/yanivalfasy/aniktest2')
# // Zoom to a location.
# Map.setCenter(0,50, 3); // Center on
# // Display the image on the map.
# // Map.addLayer(popden,{color:'blue'},'popden');

# var empty = ee.Image().byte();
# //Change to Dark theme
# var baseChange = [{featureType: 'all', stylers: [{invert_lightness: true}]}];
# Map.setOptions('baseChange', {'baseChange': baseChange});
# //convert to filled polygons
# var eurobeams_filled = ee.Geometry.MultiPolygon(eurobeams.geometry().coordinates())
# var usbeams_filled = ee.Geometry.MultiPolygon(usbeams.geometry().coordinates())

# Map.addLayer(eurobeams_filled,{color: 'lime',strokeWidth:2},'eurobeams',true,.2)
# Map.addLayer(usbeams_filled,{color: 'yellow',strokeWidth:2},'usbeams',true,.2)
# // image.merge()
# // var areaImage = image.multiply(table2
# var sat_coverage_eurobeams = eurobeams_filled.area().divide(1e6)
# print(sat_coverage_eurobeams,'km')


# // COUNTRY LIBRARY
# var country_co = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
#   .filter(ee.Filter.eq('country_co', 'FR'));
  
# var nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
#   .select('stable_lights')
#   .clipToCollection(country_co);

# var zones = nl2012.gt(30).add(nl2012.gt(55)).add(nl2012.gt(62));
# zones = zones.updateMask(zones.neq(0));


# Map.addLayer(zones, {min: 1, max: 3, palette: ['0000FF', '00FF00', 'FF0000']}, 'raster');
# var display = ee.Image(0).updateMask(0).paint(vectors, '000000', 3);
# Map.addLayer(display, {palette: '000000'}, 'vectors');
# var interGeo = eurobeams_filled.union(zones.geometry(), 100);

# // ADD LAYER 

# Map.addLayer(interGeo,{},'intersect')
# var beamIntersection = eurobeams_filled.intersection({'right': vectors, 'maxError': 1e8});
# var total_pop_intersect = beamIntersection.area().divide(100)
# print(total_pop_intersect)
# var percent_pop  =  total_pop_intersect.divide(sat_coverage_eurobeams)
# print(percent_pop,'%')



In [ ]:
# popden = ee.Image('users/yanivalfasy/ppp_2020_1km_Aggregated')
# eurobeams = ee.FeatureCollection('users/yanivalfasy/kasat')
# usbeams_lg = ee.FeatureCollection('users/yanivalfasy/aniktest2')
# usbeams_sm = ee.FeatureCollection('users/yanivalfasy/aniktest2')

# Set visualization parameters.
# vis_params = {
#   'min': 0,
#   'max': 4000,
#   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}